# Purpose

Work on early routing code to enable (primarily) some useful screenshots for the design prototype but also to test out some theories.

# Imports

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from rich import print
import pandas as pd

# Electrify America in Springfield, VA mall parking lot
TEST_LOCATION = 252784

from dotenv import load_dotenv
load_dotenv(override=True)

from evlens.logs import setup_logger
logger = setup_logger("Notebook-2.0")
logger.info("TEST!")

2024-08-01_T17_36_57EDT: INFO (Notebook-2.0:L16) - TEST!


In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Constants

In [4]:
WHITEHOUSE_COORDS = (38.8964457,-77.035921) # for funsies
BOSTON_COORDS = (42.3144474,-71.0526845)
CLEVELAND_COORDS = (41.4973681,-82.0176764)

# Trying out Folium

I hear it's a very capable high-quality mapping tool, so let's give it a whirl! We'll do everything initially with OpenStreetMap and open source routing to get a feel for things (may eventually switch to Google Routes API).

## Starting Point Map

In [5]:
from evlens.visualization.folium_tools import get_single_point
import folium

single_point_map = get_single_point(WHITEHOUSE_COORDS, 'White House', include_zoom_widget=False, starting_zoom=16)
single_point_map

## Visualizing a Route

In [6]:
from evlens.data.routing import get_openrouting_route

route, route_coords = get_openrouting_route(WHITEHOUSE_COORDS, CLEVELAND_COORDS, reverse_coordinates=True)
len(route_coords)

4324

## Find stations near route

This is the simple version that uses NREL's nearest-to-route API. We'll eventually use our own approach and our own station data of course.

In [7]:
# Find NREL-supported charging stations near route
from evlens.data.nrel_api import AFDCStationsByRoute
from shapely.geometry import LineString, Point
from typing import Tuple

# Get WKT-structured route
route_linestring = LineString(route['features'][0]['geometry']['coordinates'])
route_linestring_wkt = str(route_linestring)

afdc = AFDCStationsByRoute()
df = afdc.get_stations_near_route(route_linestring_wkt, 5, ev_connector_type='J1772COMBO') # 0.25 miles from initial route
# df = df[df['ev_network'] != 'Tesla']
df = df[df['ev_dc_fast_num'] > 2]# Require that there be more than 2 plugs, due to reliability concerns

# Find the distance from starting point to a station, "as the crow flies" is good enough for now
# Note that this is in units of lat/long, but fine for ranking/sorting purposes
def calculate_distances(start_coords: Tuple[float, float], end_coords: Tuple[float, float]) -> float:
    start_point = Point(tuple(reversed(start_coords)))
    end_point = Point(tuple(reversed(end_coords)))
    return start_point.distance(end_point)

df['distance_from_start'] = df.apply(lambda row: calculate_distances(WHITEHOUSE_COORDS, row[['latitude', 'longitude']]), axis=1)

df['index'] = df.index

df.info()
df['ev_network'].value_counts()

2024-08-01_T17_38_05EDT: INFO (evlens.data.nrel_api:L190) - 112 total records found, comprised of 241 plugs


<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, 1 to 110
Data columns (total 77 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   access_code                 29 non-null     object 
 1   access_days_time            29 non-null     object 
 2   access_detail_code          0 non-null      object 
 3   cards_accepted              1 non-null      object 
 4   date_last_confirmed         28 non-null     object 
 5   expected_date               0 non-null      object 
 6   fuel_type_code              29 non-null     object 
 7   groups_with_access_code     29 non-null     object 
 8   id                          29 non-null     int64  
 9   maximum_vehicle_class       2 non-null      object 
 10  open_date                   29 non-null     object 
 11  owner_type_code             2 non-null      object 
 12  restricted_access           2 non-null      object 
 13  status_code                 29 non-null  

/var/folders/98/fhwnl49n19l_xywxzghbm4jm0000gn/T/ipykernel_56860/1999377.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  end_point = Point(tuple(reversed(end_coords)))
/var/folders/98/fhwnl49n19l_xywxzghbm4jm0000gn/T/ipykernel_56860/1999377.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  end_point = Point(tuple(reversed(end_coords)))
/var/folders/98/fhwnl49n19l_xywxzghbm4jm0000gn/T/ipykernel_56860/1999377.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value

ev_network
Electrify America    13
eVgo Network         11
Non-Networked         1
EV Connect            1
CHARGESMART_EV        1
Tesla                 1
SHELL_RECHARGE        1
Name: count, dtype: int64

In [14]:
from evlens.visualization.folium_tools import plot_route, add_stations_to_map

map = plot_route(WHITEHOUSE_COORDS, CLEVELAND_COORDS, 'Washington, DC', 'Cleveland, OH', route_coordinates=route_coords, include_zoom_widget=False, starting_zoom=6)

# Require that charge be sufficiently far from starting point (due to home charging)


map = add_stations_to_map(
    # df[df['distance'] > 3].groupby('state', as_index=False).last(), # Get well-spaced, off-the-naive-route stations
    df,
    map,
    df.index,
    tooltip=True,
    networks_to_include='all')
map

2024-08-01_T17_43_53EDT: INFO (evlens.visualization.folium_tools:L193) - Counts of networks along the route: ev_network
Electrify America    13
eVgo Network         11
Non-Networked         1
EV Connect            1
CHARGESMART_EV        1
Tesla                 1
SHELL_RECHARGE        1
Name: count, dtype: int64


In [16]:
# NREL station IDs
stations_chosen_index = [
    167187, # Bedford EA
    183555, # Cranberry Twshp EA
]

df['reliability_score'] = 0.1
df.loc[df['id'].isin(stations_chosen_index), 'reliability_score'] = [1.0, 0.5]

In [17]:
# Draw a circle indicating max distance from start that car can go (assume 209 miles here)
MAX_RANGE = 209

In [18]:
from evlens.visualization.folium_tools import plot_route, add_stations_to_map

map = plot_route(WHITEHOUSE_COORDS, CLEVELAND_COORDS, 'Washington, DC', 'Cleveland, OH', route_coordinates=route_coords, include_zoom_widget=False, starting_zoom=6)

# Require that charge be sufficiently far from starting point (due to home charging)


map = add_stations_to_map(
    # df[df['distance'] > 3].groupby('state', as_index=False).last(), # Get well-spaced, off-the-naive-route stations
    df,
    map,
    stations_chosen_index,
    tooltip=True,
    networks_to_include='all')
map

2024-08-01_T17_45_49EDT: INFO (evlens.visualization.folium_tools:L193) - Counts of networks along the route: ev_network
Electrify America    13
eVgo Network         11
Non-Networked         1
EV Connect            1
CHARGESMART_EV        1
Tesla                 1
SHELL_RECHARGE        1
Name: count, dtype: int64


In [21]:
from evlens.visualization.folium_tools import plot_route, add_stations_to_map

map = plot_route(
    WHITEHOUSE_COORDS,
    CLEVELAND_COORDS,
    'Washington, DC',
    'Cleveland, OH',
    route_coordinates=route_coords,
    include_zoom_widget=False,
    starting_zoom=6,
    map_tiles='OpenTopoMap'
)

# Require that charge be sufficiently far from starting point (due to home charging)


map = add_stations_to_map(
    # df[df['distance'] > 3].groupby('state', as_index=False).last(), # Get well-spaced, off-the-naive-route stations
    df,
    map,
    stations_chosen_index,
    tooltip=True,
    networks_to_include='all'
)
map

2024-08-01_T17_51_40EDT: INFO (evlens.visualization.folium_tools:L193) - Counts of networks along the route: ev_network
Electrify America    13
eVgo Network         11
Non-Networked         1
EV Connect            1
CHARGESMART_EV        1
Tesla                 1
SHELL_RECHARGE        1
Name: count, dtype: int64


In [26]:
df['distance'].describe()

count    28.000000
mean      1.876203
std       1.701833
min       0.049390
25%       0.371487
50%       1.328455
75%       3.335780
max       4.679950
Name: distance, dtype: float64

In [ ]:
from evlens.visualization.folium_tools import plot_route, add_stations_to_map

map = plot_route(WHITEHOUSE_COORDS, CLEVELAND_COORDS, 'Washington, DC', 'Cleveland, OH', route_coordinates=route_coords, include_zoom_widget=False, starting_zoom=6)

# Require that charge be sufficiently far from starting point (due to home charging)


map = add_stations_to_map(
    # df[df['distance'] > 3].groupby('state', as_index=False).last(), # Get well-spaced, off-the-naive-route stations
    df[df['distance_from_start'] > 1],
    map,
    tooltip=True,
    networks_to_include='all')
map

2024-07-31_T16_10_06EDT: INFO (evlens.visualization.folium_tools:L154) - Counts of networks along the route: ev_network
eVgo Network         4
Electrify America    3
Non-Networked        1
EV Connect           1
CHARGESMART_EV       1
Tesla                1
SHELL_RECHARGE       1
Name: count, dtype: int64


In [27]:
df[df['distance'] > 3].groupby('state').last()

,access_code,access_days_time,access_detail_code,cards_accepted,date_last_confirmed,expected_date,fuel_type_code,groups_with_access_code,id,maximum_vehicle_class,open_date,owner_type_code,restricted_access,status_code,federal_funding_types,facility_type,station_name,station_phone,updated_at,geocode_status,latitude,longitude,city,country,intersection_directions,plus4,street_address,zip,bd_blends,cng_dispenser_num,cng_fill_type_code,cng_has_rng,cng_psi,cng_renewable_source,cng_total_compression,cng_total_storage,cng_vehicle_class,e85_blender_pump,e85_other_ethanol_blends,ev_connector_types,ev_dc_fast_num,ev_level1_evse_num,ev_level2_evse_num,ev_network,ev_network_web,ev_other_evse,ev_pricing,ev_renewable_source,ev_workplace_charging,hy_is_retail,hy_pressures,hy_standards,hy_status_link,lng_has_rng,lng_renewable_source,lng_vehicle_class,lpg_nozzle_types,lpg_primary,ng_fill_type_code,ng_psi,ng_vehicle_class,rd_blended_with_biodiesel,rd_blends,rd_blends_fr,rd_max_biodiesel_level,nps_unit_name,access_days_time_fr,intersection_directions_fr,bd_blends_fr,groups_with_access_code_fr,ev_pricing_fr,ev_network_ids,distance,distance_km
state,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DC,public,24 hours daily,None,None,2024-07-30,None,ELEC,Public,163628,None,2020-06-25,None,None,E,None,None,"Walmart 5968 - Georgia Ave, DC",833-632-2778,2024-07-30T02:35:55Z,GPS,38.961480,-77.026790,Washington,US,None,None,5929 Georgia Ave NW,20011,None,None,None,None,None,None,None,None,None,None,None,"[CHADEMO, J1772, J1772COMBO]",3,None,1.0,Electrify America,https://www.electrifyamerica.com/,None,None,None,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Public,None,"{'station': ['200225'], 'posts': ['200225-01',...",4.12152,6.63294
MD,public,24 hours daily,None,None,2024-07-30,None,ELEC,Public,206526,None,2022-02-17,None,None,E,None,None,"Westfield Wheaton (Siver Spring, MD)",833-632-2778,2024-07-30T02:36:09Z,GPS,39.038100,-77.054250,Silver Spring,US,None,None,11160 Veirs Mill Rd,20902,None,None,None,None,None,None,None,None,None,None,None,"[CHADEMO, J1772COMBO]",6,None,NaN,Electrify America,https://www.electrifyamerica.com/,None,None,None,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Public,None,"{'station': ['210248'], 'posts': ['210248-01',...",4.67995,7.53165
OH,public,24 hours daily,None,None,2024-07-30,None,ELEC,Public,124685,None,2019-06-25,None,None,E,None,None,Sheffield Crossing Station,833-632-2778,2024-07-30T02:36:26Z,GPS,41.426890,-82.079730,Sheffield,US,None,None,5231 Detroit Rd,44054,None,None,None,None,None,None,None,None,None,None,None,[J1772COMBO],4,None,NaN,Electrify America,https://www.electrifyamerica.com/,None,None,None,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Public,None,"{'station': ['100236'], 'posts': ['100236-01',...",3.14661,5.06398
PA,public,24 hours daily,None,None,2024-07-29,None,ELEC,Public,228659,None,2022-07-02,None,None,E,None,None,McCandless Crossing,877-455-3833,2024-07-29T23:59:25Z,GPS,40.570994,-80.022932,Pittsburgh,US,None,None,9150 Covenant Ave,15237,None,None,None,None,None,None,None,None,None,None,None,"[CHADEMO, J1772COMBO]",4,None,NaN,eVgo Network,https://www.evgo.com/,None,None,None,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Public,None,"{'station': ['44162'], 'posts': ['88440', '884...",3.90329,6.28174
VA,public,24 hours daily,None,None,2024-07-29,None,ELEC,Public,228713,None,2021-08-26,None,None,E,None,None,Safeway Mclean #1920,877-455-3833,2024-07-29T23:59:49Z,GPS,38.922475,-77.201390,McLean,US,None,None,1688 Anderson Rd,22102,None,None,None,None,None,None,None,None,None,None,None,"[CHADEMO, J1772COMBO]",4,None,NaN,eVgo Network,https://www.evgo.com/,None,None,None,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None